In [10]:
using DataFrames, CSV, JuMP, Gurobi, LinearAlgebra, Random, Printf, StatsBase, CategoricalArrays, Plots, StatsPlots, Distributions

In [11]:
# # add actual data here
# xgb_preds = CSV.read("C:\\Users\\shann\\Documents\\GitHub\\15095-project\\notebooks\\xgb_cv_pred.csv",DataFrame)
# rf_preds = CSV.read("C:\\Users\\shann\\Documents\\GitHub\\15095-project\\notebooks\\rf_pred.csv",DataFrame)
# lgbm_preds = CSV.read("C:\\Users\\shann\\Documents\\GitHub\\15095-project\\notebooks\\lgbm_cv_pred.csv",DataFrame);
# logreg_preds = CSV.read("C:\\Users\\shann\\Documents\\GitHub\\15095-project\\notebooks\\logreg_pred.csv",DataFrame);
# new_xgb_preds = CSV.read("C:\\Users\\shann\\Documents\\GitHub\\15095-project\\notebooks\\new_xgb_pred.csv",DataFrame);
# returns = CSV.read("C:\\Users\\shann\\Documents\\GitHub\\15095-project\\data\\group_stage_betting_odds_final_cleaned.csv",DataFrame);

# # add actual data here
new_xgb_preds = CSV.read("/Users/yenhann/Documents/MIT/15.095 Machine Learning/15095-project/notebooks/new_xgb_pred.csv",DataFrame);
returns = CSV.read("/Users/yenhann/Documents/MIT/15.095 Machine Learning/15095-project/data/group_stage_betting_odds_final_cleaned.csv",DataFrame);
real_outcomes = CSV.read("/Users/yenhann/Documents/MIT/15.095 Machine Learning/15095-project/data/group_stage_mapped_outcomes.csv",DataFrame);

In [12]:
# check if each row of new_xgb_preds and returns are the same
for i in 1:size(new_xgb_preds,1)
    if new_xgb_preds[i,1] != returns[i,1] || new_xgb_preds[i,2] != returns[i,2]
        println("XGB and Returns not the same")
    end
end

# check if each row of new_xgb_preds and real_outcomes are the same
for i in 1:size(new_xgb_preds,1)
    if new_xgb_preds[i,1] != real_outcomes[i,1] || new_xgb_preds[i,2] != real_outcomes[i,2]
        println("XGB and Real Outcomes not the same")
    end
end

# check if returns and real_outcomes are the same
for i in 1:size(returns,1)
    if returns[i,1] != real_outcomes[i,1] || returns[i,2] != real_outcomes[i,2]
        println("Returns and Real Outcomes not the same")
    end
end

In [13]:
# returns
returns

Row,home_team,away_team,home_win_earnings_per_dollar,draw_earnings_per_dollar,away_win_earnings_per_dollar,implied_probability_of_home_win,implied_probability_of_draw,implied_probability_of_away_win
,String15,String15,Float64,Float64,Float64,Float64,Float64,Float64
1,Argentina,Mexico,0.549451,2.95,6.12,0.64539,0.253165,0.140449
2,Argentina,Saudi Arabia,0.120048,8.21,24.52,0.892819,0.108578,0.039185
3,Australia,Denmark,6.38,3.88,0.429185,0.135501,0.204918,0.6997
4,Belgium,Canada,0.900901,2.45,3.51,0.526066,0.289855,0.221729
5,Belgium,Morocco,0.581395,3.21,4.92,0.632353,0.23753,0.168919
6,Brazil,Serbia,0.5,3.28,6.6,0.666667,0.233645,0.131579
7,Brazil,Switzerland,0.469484,3.57,6.23,0.680511,0.218818,0.138313
8,Cameroon,Brazil,7.93,4.32,0.359712,0.111982,0.18797,0.73545
9,Cameroon,Serbia,4.3,2.77,0.70922,0.188679,0.265252,0.585062


In [14]:
# for each row in real_outcomes, if :Mapped_Outcomes = 1, then save value of home_win_earnings_per_dollar from returns
# if :Mapped_Outcomes = 2, then save value of draw_earnings_per_dollar from returns
# if :Mapped_Outcomes = 3, then save value of away_win_earnings_per_dollar from returns
real_earnings_per_dollar = zeros(size(real_outcomes,1),2)

for i in 1:size(real_outcomes,1)
    if real_outcomes[i,:Mapped_Outcomes] == 1
        real_earnings_per_dollar[i,1] = 1
        real_earnings_per_dollar[i,2] = returns[i,:home_win_earnings_per_dollar]
    elseif real_outcomes[i,:Mapped_Outcomes] == 2
        real_earnings_per_dollar[i,1] = 2
        real_earnings_per_dollar[i,2] = returns[i,:draw_earnings_per_dollar]
    elseif real_outcomes[i,:Mapped_Outcomes] == 3
        real_earnings_per_dollar[i,1] = 3
        real_earnings_per_dollar[i,2] = returns[i,:away_win_earnings_per_dollar]
    end
end

In [15]:
function FIFA2022_GS_Bettings_GodsEye(R, B, K)
    
    # define model
    model = Model(Gurobi.Optimizer)

    # define parameters and decision variables
    m = 48             # 48 group stage matches

    @variable(model, x[i=1:m] >= 0)     # x[i] is the amount of money bet on match knowing its outcome
    @variable(model, z[i=1:m], Bin)     # z[i] is whether you choose to bet on match i or not

    # define objective: maximising expected total returns across all matches
    @objective(model, Max, sum(x[i]*R[i] for i=1:m))

    # # constraints
    # 1) x[i] <= B*z[i] for all i
    @constraint(model, [i=1:m], x[i] <= (B/20)*z[i])

    # 2) Budget constraints
    @constraint(model, sum(x[i] for i=1:m) <= B)

    # 3) Must bet on at least K matches
    @constraint(model, sum(z[i] for i=1:m) >= K)

    # 4) if z = 1, then x > 0
    @constraint(model, [i=1:m], x[i] >= 1*z[i])

    # solve model
    optimize!(model)

    # get optimal bet amounts (x) and how you bet (z)
    opt_val = objective_value(model)
    x_opt = value.(x)
    z_opt = value.(z)

    return opt_val, x_opt, z_opt
end

FIFA2022_GS_Bettings_GodsEye (generic function with 1 method)

In [16]:
exp_rev, x_opt, z_opt = FIFA2022_GS_Bettings_GodsEye(real_earnings_per_dollar[:,2], 5000, 5);

Set parameter Username
Academic license - for non-commercial use only - expires 2023-08-19
Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (mac64[x86])
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 98 rows, 96 columns and 288 nonzeros
Model fingerprint: 0xf36524e2
Variable types: 48 continuous, 48 integer (48 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+02]
  Objective range  [1e-01, 2e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [5e+00, 5e+03]
Found heuristic solution: objective 2634.9660635
Presolve time: 0.00s
Presolved: 98 rows, 96 columns, 288 nonzeros
Variable types: 0 continuous, 96 integer (48 binary)
Found heuristic solution: objective 2747.1144854

Root relaxation: objective 2.493750e+04, 24 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0 

In [17]:
# print the expected profit
println("Expected profit: ", exp_rev-sum(x_opt))

Expected profit: 19937.5


In [18]:
# get indices where z_opt = 1
betting_indices = findall(x -> x == 1, z_opt)

# output rows of returns corresponding to betting_indices
betting_returns = returns[betting_indices,:]

# per match earnings
per_match_earnings = zeros(length(x_opt))

for i = 1:length(x_opt)
    per_match_earnings[i] = x_opt[i]*real_earnings_per_dollar[i,2]
end

# take the betting indices and get the corresponding per_match_earnings
betting_earnings = per_match_earnings[betting_indices]

# append betting_earnings to betting_returns
betting_returns[!,:betting_earnings] = betting_earnings

# THE ARGENTINA V. SAUDI ARABIA CASH COW
betting_returns[!,[1,2,3,4,5,9]]

Row,home_team,away_team,home_win_earnings_per_dollar,draw_earnings_per_dollar,away_win_earnings_per_dollar,betting_earnings
,String15,String15,Float64,Float64,Float64,Float64
1,Argentina,Saudi Arabia,0.120048,8.21,24.52,6130.0
2,Australia,Denmark,6.38,3.88,0.429185,1595.0
3,Belgium,Morocco,0.581395,3.21,4.92,1230.0
4,Cameroon,Brazil,7.93,4.32,0.359712,1982.5
5,Cameroon,Serbia,4.3,2.77,0.70922,692.5
6,Croatia,Belgium,2.02,2.41,1.44,602.5
7,Denmark,Tunisia,0.581395,2.92,5.57,730.0
8,England,USA,0.380228,3.65,9.19,912.5
9,Germany,Japan,0.45045,3.84,6.28,1570.0
